In [ ]:
from flashml import plot_dist
import random
x1 = [random.randint(0, 5) for _ in range(10)]
x2 = [random.randint(0, 200) for _x in range(1000)]
x3 = [None, *x2, None]
x4= [random.random() for _ in range(1000)]
x5 = [None, *x4, None]
x6 = ["hi there"  , "jonanatha", "hi there", "hi there "]
x7 = [None, *x6]
x8 = {"daniel": 10, "james": 10}
plot_dist(x1)
plot_dist(x2)
plot_dist(x3)
plot_dist(x4)
plot_dist(x5)
plot_dist(x6)
plot_dist(x7)       
plot_dist(x8)  

In [ ]:
from flashml import host_mlflow


host_mlflow()

In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
import json
import difflib
from typing import Optional, List, Tuple
import re

# Try to import tkinterdnd2 for drag and drop support
try:
    from tkinterdnd2 import DND_FILES, TkinterDnD
    DRAG_DROP_AVAILABLE = True
except ImportError:
    DRAG_DROP_AVAILABLE = False
    TkinterDnD = tk

class FileViewerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("File Viewer & Text Comparator")
        self.root.geometry("500x300")
        self.center_window(500, 300)
        
        # Remove default window decorations
        self.root.overrideredirect(True)
        
        # Variables
        self.dark_mode = True
        self.current_file = None
        self.df = None
        self.search_positions = []
        self.current_search_index = -1
        self.original_values = {}
        self.drag_drop_enabled = DRAG_DROP_AVAILABLE
        
        # Colors
        self.update_colors()
        
        # Show startup screen
        self.show_startup_screen()
        
        # Bind dragging for custom window
        self.root.bind('<Button-1>', self.start_move)
        self.root.bind('<B1-Motion>', self.on_move)
        
    def update_colors(self):
        if self.dark_mode:
            self.bg_color = "#1e1e1e"
            self.fg_color = "#d4d4d4"
            self.button_bg = "#2d2d30"
            self.button_fg = "#cccccc"
            self.entry_bg = "#3c3c3c"
            self.entry_fg = "#d4d4d4"
            self.table_bg = "#252526"
            self.table_fg = "#d4d4d4"
            self.header_bg = "#2d2d30"
            self.selected_bg = "#094771"
            self.highlight_cell = "#264f3d"
            self.highlight_text = "#3d6b4f"
            self.col_colors = ["#2a2a2a", "#252525", "#2f2f2f"]
        else:
            self.bg_color = "#ffffff"
            self.fg_color = "#000000"
            self.button_bg = "#f3f3f3"
            self.button_fg = "#000000"
            self.entry_bg = "#ffffff"
            self.entry_fg = "#000000"
            self.table_bg = "#ffffff"
            self.table_fg = "#000000"
            self.header_bg = "#e0e0e0"
            self.selected_bg = "#cce8ff"
            self.highlight_cell = "#c6efce"
            self.highlight_text = "#92d050"
            self.col_colors = ["#fafafa", "#f5f5f5", "#f0f0f0"]
    
    def center_window(self, width, height):
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()
        x = (screen_width - width) // 2
        y = (screen_height - height) // 2
        self.root.geometry(f"{width}x{height}+{x}+{y}")
    
    def start_move(self, event):
        if hasattr(self, 'title_bar') and event.widget == self.title_bar:
            self.x = event.x
            self.y = event.y
    
    def on_move(self, event):
        if hasattr(self, 'title_bar') and event.widget == self.title_bar:
            deltax = event.x - self.x
            deltay = event.y - self.y
            x = self.root.winfo_x() + deltax
            y = self.root.winfo_y() + deltay
            self.root.geometry(f"+{x}+{y}")
    
    def show_startup_screen(self):
        self.clear_window()
        
        # Title bar
        self.create_title_bar("File Viewer & Text Comparator", show_back=False)
        
        # Main content
        content_frame = tk.Frame(self.root, bg=self.bg_color)
        content_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)
        
        title = tk.Label(content_frame, text="Choose Mode", font=("Arial", 18, "bold"),
                        bg=self.bg_color, fg=self.fg_color)
        title.pack(pady=30)
        
        # View File Button
        view_btn = tk.Button(content_frame, text="View File", font=("Arial", 14),
                            bg=self.button_bg, fg=self.button_fg,
                            activebackground=self.selected_bg,
                            relief=tk.FLAT, padx=30, pady=15,
                            command=self.show_file_drop_screen)
        view_btn.pack(pady=10, fill=tk.X)
        
        # Compare Texts Button
        compare_btn = tk.Button(content_frame, text="Compare Texts", font=("Arial", 14),
                               bg=self.button_bg, fg=self.button_fg,
                               activebackground=self.selected_bg,
                               relief=tk.FLAT, padx=30, pady=15,
                               command=self.show_text_compare_screen)
        compare_btn.pack(pady=10, fill=tk.X)
    
    def show_file_drop_screen(self):
        self.clear_window()
        
        # Title bar
        self.create_title_bar("View File", show_back=True)
        
        # Drag and drop area
        drop_frame = tk.Frame(self.root, bg=self.bg_color)
        drop_frame.pack(fill=tk.BOTH, expand=True, padx=40, pady=40)
        
        drop_area = tk.Frame(drop_frame, bg=self.button_bg, relief=tk.SOLID, bd=2)
        drop_area.pack(fill=tk.BOTH, expand=True)
        
        icon_label = tk.Label(drop_area, text="[FILE]", font=("Arial", 48),
                             bg=self.button_bg, fg=self.fg_color)
        icon_label.pack(pady=20)
        
        # Show different text based on drag-drop availability
        if self.drag_drop_enabled:
            instruction_text = "Drag and drop file here or click to browse\n\nSupported: CSV, XLS, XLSX, JSONL"
        else:
            instruction_text = "Click to browse and select file\n\nSupported: CSV, XLS, XLSX, JSONL\n\n(Install tkinterdnd2 for drag-drop: pip install tkinterdnd2)"
        
        text_label = tk.Label(drop_area, text=instruction_text,
                             font=("Arial", 11), bg=self.button_bg, fg=self.fg_color)
        text_label.pack(pady=10)
        
        # Enable drag and drop if available
        if self.drag_drop_enabled:
            drop_area.drop_target_register(DND_FILES)
            drop_area.dnd_bind('<<Drop>>', self.drop_file)
            icon_label.drop_target_register(DND_FILES)
            icon_label.dnd_bind('<<Drop>>', self.drop_file)
            text_label.drop_target_register(DND_FILES)
            text_label.dnd_bind('<<Drop>>', self.drop_file)
        
        # Also allow clicking
        drop_area.bind("<Button-1>", lambda e: self.select_file())
        icon_label.bind("<Button-1>", lambda e: self.select_file())
        text_label.bind("<Button-1>", lambda e: self.select_file())
    
    def drop_file(self, event):
        """Handle file drop event"""
        file_path = event.data
        # Clean up the file path (remove braces if present)
        if file_path.startswith('{') and file_path.endswith('}'):
            file_path = file_path[1:-1]
        self.load_file(file_path)
    
    def select_file(self):
        file_path = filedialog.askopenfilename(
            title="Select a file",
            filetypes=[
                ("All supported", "*.csv *.xls *.xlsx *.jsonl"),
                ("CSV files", "*.csv"),
                ("Excel files", "*.xls *.xlsx"),
                ("JSONL files", "*.jsonl")
            ]
        )
        if file_path:
            self.load_file(file_path)
    
    def load_file(self, file_path):
        try:
            if file_path.endswith('.csv'):
                self.df = pd.read_csv(file_path)
            elif file_path.endswith(('.xls', '.xlsx')):
                self.df = pd.read_excel(file_path)
            elif file_path.endswith('.jsonl'):
                data = []
                with open(file_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        data.append(json.loads(line))
                self.df = pd.DataFrame(data)
            
            self.current_file = file_path
            self.show_file_viewer()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load file:\n{str(e)}")
    
    def show_file_viewer(self):
        self.clear_window()
        self.root.state('zoomed')  # Fullscreen
        
        # Title bar
        self.create_title_bar(f"Viewing: {self.current_file.split('/')[-1]}", show_back=True)
        
        # Toolbar
        toolbar = tk.Frame(self.root, bg=self.button_bg, height=40)
        toolbar.pack(fill=tk.X, padx=5, pady=5)
        
        # Save button
        save_btn = tk.Button(toolbar, text="Save", bg=self.button_bg, fg=self.button_fg,
                            relief=tk.FLAT, padx=10, command=self.save_file)
        save_btn.pack(side=tk.LEFT, padx=2)
        
        # Save copy button
        save_copy_btn = tk.Button(toolbar, text="Save Copy", bg=self.button_bg, fg=self.button_fg,
                                 relief=tk.FLAT, padx=10, command=self.save_file_copy)
        save_copy_btn.pack(side=tk.LEFT, padx=2)
        
        # Export button
        export_btn = tk.Button(toolbar, text="Export", bg=self.button_bg, fg=self.button_fg,
                              relief=tk.FLAT, padx=10, command=self.export_file)
        export_btn.pack(side=tk.LEFT, padx=2)
        
        # Separator
        tk.Frame(toolbar, bg=self.fg_color, width=2).pack(side=tk.LEFT, fill=tk.Y, padx=5)
        
        # Search entry
        tk.Label(toolbar, text="Search:", bg=self.button_bg, fg=self.fg_color).pack(side=tk.LEFT, padx=2)
        self.search_var = tk.StringVar()
        self.search_var.trace('w', self.on_search_change)
        search_entry = tk.Entry(toolbar, textvariable=self.search_var, bg=self.entry_bg,
                               fg=self.entry_fg, relief=tk.FLAT, width=20)
        search_entry.pack(side=tk.LEFT, padx=2)
        
        # Replace frame (initially hidden)
        self.replace_frame = tk.Frame(toolbar, bg=self.button_bg)
        
        tk.Label(self.replace_frame, text="Replace:", bg=self.button_bg, fg=self.fg_color).pack(side=tk.LEFT, padx=2)
        self.replace_var = tk.StringVar()
        self.replace_var.trace('w', self.on_replace_change)
        self.replace_entry = tk.Entry(self.replace_frame, textvariable=self.replace_var,
                                      bg=self.entry_bg, fg=self.entry_fg, relief=tk.FLAT, width=15)
        self.replace_entry.pack(side=tk.LEFT, padx=2)
        
        self.replace_next_btn = tk.Button(self.replace_frame, text="Replace Next",
                                         bg=self.button_bg, fg=self.button_fg,
                                         relief=tk.FLAT, state=tk.DISABLED,
                                         command=self.replace_next)
        self.replace_next_btn.pack(side=tk.LEFT, padx=2)
        
        self.replace_all_btn = tk.Button(self.replace_frame, text="Replace All",
                                        bg=self.button_bg, fg=self.button_fg,
                                        relief=tk.FLAT, state=tk.DISABLED,
                                        command=self.replace_all)
        self.replace_all_btn.pack(side=tk.LEFT, padx=2)
        
        # Separator
        tk.Frame(toolbar, bg=self.fg_color, width=2).pack(side=tk.LEFT, fill=tk.Y, padx=5)
        
        # Dark/Light mode toggle
        mode_text = "Light Mode" if self.dark_mode else "Dark Mode"
        mode_btn = tk.Button(toolbar, text=mode_text, bg=self.button_bg, fg=self.button_fg,
                           relief=tk.FLAT, padx=10, command=self.toggle_mode)
        mode_btn.pack(side=tk.LEFT, padx=2)
        
        # Table frame with scrollbars
        table_frame = tk.Frame(self.root, bg=self.bg_color)
        table_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Create Treeview
        self.tree = ttk.Treeview(table_frame, show='tree headings')
        
        # Scrollbars
        vsb = ttk.Scrollbar(table_frame, orient="vertical", command=self.tree.yview)
        hsb = ttk.Scrollbar(table_frame, orient="horizontal", command=self.tree.xview)
        self.tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)
        
        vsb.pack(side=tk.RIGHT, fill=tk.Y)
        hsb.pack(side=tk.BOTTOM, fill=tk.X)
        self.tree.pack(fill=tk.BOTH, expand=True)
        
        # Configure style
        style = ttk.Style()
        style.theme_use('default')
        
        if self.dark_mode:
            style.configure("Treeview",
                          background=self.table_bg,
                          foreground=self.table_fg,
                          fieldbackground=self.table_bg,
                          borderwidth=0)
            style.configure("Treeview.Heading",
                          background=self.header_bg,
                          foreground=self.fg_color,
                          borderwidth=1)
            style.map('Treeview', background=[('selected', self.selected_bg)])
        else:
            style.configure("Treeview",
                          background=self.table_bg,
                          foreground=self.table_fg,
                          fieldbackground=self.table_bg,
                          borderwidth=1)
            style.configure("Treeview.Heading",
                          background=self.header_bg,
                          foreground=self.fg_color,
                          borderwidth=1)
            style.map('Treeview', background=[('selected', self.selected_bg)])
        
        # Setup columns
        columns = ['#'] + list(self.df.columns)
        self.tree['columns'] = columns
        
        # Configure index column
        self.tree.column('#0', width=0, stretch=tk.NO)
        self.tree.heading('#', text='#')
        self.tree.column('#', width=50, anchor=tk.CENTER)
        
        # Configure data columns with heuristic sizing
        for i, col in enumerate(self.df.columns):
            self.tree.heading(col, text=col)
            width = self.calculate_column_width(col, i)
            self.tree.column(col, width=width, anchor=tk.W)
            
            # Add alternating column colors using tags
            tag_name = f"col_{i % len(self.col_colors)}"
            self.tree.tag_configure(tag_name, background=self.col_colors[i % len(self.col_colors)])
        
        # Insert data
        for idx, row in self.df.iterrows():
            values = [idx] + [self.wrap_text(str(val), 50) for val in row]
            tags = tuple([f"col_{i % len(self.col_colors)}" for i in range(len(self.df.columns))])
            self.tree.insert('', tk.END, values=values, tags=tags)
        
        # Enable editing
        self.tree.bind('<Double-1>', self.on_double_click)
        
        # Configure tags for search highlighting
        self.tree.tag_configure('highlight_cell', background=self.highlight_cell)
        self.tree.tag_configure('highlight_text', background=self.highlight_text)
    
    def calculate_column_width(self, col_name, col_idx):
        """Heuristic to calculate column width based on content"""
        # Check column name
        if any(keyword in col_name.lower() for keyword in ['id', 'index', 'num', 'count', 'age']):
            return 80
        
        # Sample data from column
        sample_data = self.df[col_name].head(100).astype(str)
        
        # Calculate average length
        avg_len = sample_data.str.len().mean()
        max_len = sample_data.str.len().max()
        
        # Determine width
        if avg_len < 10:
            return 100
        elif avg_len < 30:
            return 150
        elif avg_len < 60:
            return 250
        else:
            return 400
    
    def wrap_text(self, text, width):
        """Wrap text for display"""
        if len(text) <= width:
            return text
        # Return full text, wrapping handled by display
        return text
    
    def on_double_click(self, event):
        """Enable in-place editing"""
        region = self.tree.identify("region", event.x, event.y)
        if region == "cell":
            column = self.tree.identify_column(event.x)
            row = self.tree.identify_row(event.y)
            
            if column == '#0' or column == '#1':  # Skip index column
                return
            
            # Get column index and value
            col_idx = int(column.replace('#', '')) - 2
            col_name = self.df.columns[col_idx]
            row_idx = int(self.tree.item(row)['values'][0])
            
            current_value = str(self.df.iloc[row_idx, col_idx])
            
            # Create entry widget for editing
            bbox = self.tree.bbox(row, column)
            if bbox:
                entry = tk.Entry(self.tree, bg=self.entry_bg, fg=self.entry_fg)
                entry.place(x=bbox[0], y=bbox[1], width=bbox[2], height=bbox[3])
                entry.insert(0, current_value)
                entry.select_range(0, tk.END)
                entry.focus()
                
                def save_edit(event=None):
                    new_value = entry.get()
                    self.df.iloc[row_idx, col_idx] = new_value
                    values = list(self.tree.item(row)['values'])
                    values[col_idx + 1] = self.wrap_text(new_value, 50)
                    self.tree.item(row, values=values)
                    entry.destroy()
                
                entry.bind('<Return>', save_edit)
                entry.bind('<FocusOut>', save_edit)
    
    def on_search_change(self, *args):
        """Handle search input change"""
        query = self.search_var.get()
        
        if query:
            if not self.replace_frame.winfo_ismapped():
                self.replace_frame.pack(side=tk.LEFT, padx=5)
            self.perform_search(query)
        else:
            if self.replace_frame.winfo_ismapped():
                self.replace_frame.pack_forget()
            self.clear_highlights()
    
    def on_replace_change(self, *args):
        """Handle replace input change"""
        replace_text = self.replace_var.get()
        
        if replace_text and self.search_positions:
            self.replace_next_btn.config(state=tk.NORMAL)
            self.replace_all_btn.config(state=tk.NORMAL)
        else:
            self.replace_next_btn.config(state=tk.DISABLED)
            self.replace_all_btn.config(state=tk.DISABLED)
    
    def perform_search(self, query):
        """Search for query in dataframe and highlight results"""
        self.clear_highlights()
        self.search_positions = []
        
        query_lower = query.lower()
        
        for item in self.tree.get_children():
            values = self.tree.item(item)['values']
            row_idx = int(values[0])
            
            for col_idx, value in enumerate(values[1:]):
                value_str = str(value).lower()
                
                if query_lower in value_str:
                    self.search_positions.append((item, col_idx))
                    # Highlight the cell
                    tags = list(self.tree.item(item)['tags'])
                    if 'highlight_cell' not in tags:
                        tags.append('highlight_cell')
                    self.tree.item(item, tags=tags)
        
        if self.search_positions:
            self.current_search_index = 0
            # Scroll to first result
            first_item = self.search_positions[0][0]
            self.tree.see(first_item)
    
    def clear_highlights(self):
        """Clear all search highlights"""
        for item in self.tree.get_children():
            tags = [tag for tag in self.tree.item(item)['tags'] if 'highlight' not in tag]
            self.tree.item(item, tags=tags)
        self.search_positions = []
        self.current_search_index = -1
    
    def replace_next(self):
        """Replace next occurrence"""
        if not self.search_positions or self.current_search_index >= len(self.search_positions):
            return
        
        search_query = self.search_var.get()
        replace_text = self.replace_var.get()
        
        item, col_idx = self.search_positions[self.current_search_index]
        values = list(self.tree.item(item)['values'])
        row_idx = int(values[0])
        
        # Get current value and replace
        current_value = str(self.df.iloc[row_idx, col_idx])
        new_value = current_value.replace(search_query, replace_text)
        
        # Update dataframe and tree
        self.df.iloc[row_idx, col_idx] = new_value
        values[col_idx + 1] = self.wrap_text(new_value, 50)
        self.tree.item(item, values=values)
        
        # Move to next
        self.current_search_index += 1
        
        # Refresh search
        self.perform_search(search_query)
    
    def replace_all(self):
        """Replace all occurrences"""
        search_query = self.search_var.get()
        replace_text = self.replace_var.get()
        
        count = 0
        for item, col_idx in self.search_positions:
            values = list(self.tree.item(item)['values'])
            row_idx = int(values[0])
            
            current_value = str(self.df.iloc[row_idx, col_idx])
            if search_query in current_value:
                new_value = current_value.replace(search_query, replace_text)
                self.df.iloc[row_idx, col_idx] = new_value
                values[col_idx + 1] = self.wrap_text(new_value, 50)
                self.tree.item(item, values=values)
                count += 1
        
        messagebox.showinfo("Replace All", f"Replaced {count} occurrences")
        self.perform_search(search_query)
    
    def save_file(self):
        """Save changes to current file"""
        if self.current_file:
            try:
                if self.current_file.endswith('.csv'):
                    self.df.to_csv(self.current_file, index=False)
                elif self.current_file.endswith(('.xls', '.xlsx')):
                    self.df.to_excel(self.current_file, index=False)
                elif self.current_file.endswith('.jsonl'):
                    with open(self.current_file, 'w', encoding='utf-8') as f:
                        for _, row in self.df.iterrows():
                            f.write(json.dumps(row.to_dict()) + '\n')
                messagebox.showinfo("Success", "File saved successfully!")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to save file:\n{str(e)}")
    
    def save_file_copy(self):
        """Save a copy of the file"""
        file_path = filedialog.asksaveasfilename(
            defaultextension=".csv",
            filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx"), ("JSONL files", "*.jsonl")]
        )
        if file_path:
            try:
                if file_path.endswith('.csv'):
                    self.df.to_csv(file_path, index=False)
                elif file_path.endswith('.xlsx'):
                    self.df.to_excel(file_path, index=False)
                elif file_path.endswith('.jsonl'):
                    with open(file_path, 'w', encoding='utf-8') as f:
                        for _, row in self.df.iterrows():
                            f.write(json.dumps(row.to_dict()) + '\n')
                messagebox.showinfo("Success", "Copy saved successfully!")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to save copy:\n{str(e)}")
    
    def export_file(self):
        """Export to different format"""
        file_path = filedialog.asksaveasfilename(
            defaultextension=".csv",
            filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx"), ("JSON files", "*.json")]
        )
        if file_path:
            try:
                if file_path.endswith('.csv'):
                    self.df.to_csv(file_path, index=False)
                elif file_path.endswith('.xlsx'):
                    self.df.to_excel(file_path, index=False)
                elif file_path.endswith('.json'):
                    self.df.to_json(file_path, orient='records', indent=2)
                messagebox.showinfo("Success", "File exported successfully!")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to export file:\n{str(e)}")
    
    def toggle_mode(self):
        """Toggle between dark and light mode"""
        self.dark_mode = not self.dark_mode
        self.update_colors()
        
        # Reload current view
        if self.df is not None:
            self.show_file_viewer()
        else:
            self.show_startup_screen()
    
    def show_text_compare_screen(self):
        self.clear_window()
        self.root.state('zoomed')  # Fullscreen
        
        # Title bar
        self.create_title_bar("Compare Texts", show_back=True)
        
        # Toolbar
        toolbar = tk.Frame(self.root, bg=self.button_bg, height=40)
        toolbar.pack(fill=tk.X, padx=5, pady=5)
        
        # Compare button
        compare_btn = tk.Button(toolbar, text="Compare", bg=self.button_bg, fg=self.button_fg,
                               relief=tk.FLAT, padx=15, command=self.compare_texts)
        compare_btn.pack(side=tk.LEFT, padx=5)
        
        # Clear button
        clear_btn = tk.Button(toolbar, text="Clear", bg=self.button_bg, fg=self.button_fg,
                             relief=tk.FLAT, padx=15, command=self.clear_text_areas)
        clear_btn.pack(side=tk.LEFT, padx=5)
        
        # Separator
        tk.Frame(toolbar, bg=self.fg_color, width=2).pack(side=tk.LEFT, fill=tk.Y, padx=5)
        
        # Dark/Light mode toggle
        mode_text = "Light Mode" if self.dark_mode else "Dark Mode"
        mode_btn = tk.Button(toolbar, text=mode_text, bg=self.button_bg, fg=self.button_fg,
                           relief=tk.FLAT, padx=10, command=self.toggle_mode)
        mode_btn.pack(side=tk.LEFT, padx=5)
        
        # Statistics label
        self.stats_label = tk.Label(toolbar, text="", bg=self.button_bg, fg=self.fg_color,
                                   font=("Arial", 10))
        self.stats_label.pack(side=tk.RIGHT, padx=10)
        
        # Main comparison frame
        compare_frame = tk.Frame(self.root, bg=self.bg_color)
        compare_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Left text area
        left_frame = tk.Frame(compare_frame, bg=self.bg_color)
        left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5)
        
        left_label = tk.Label(left_frame, text="Original Text (Left)", bg=self.bg_color,
                             fg=self.fg_color, font=("Arial", 12, "bold"))
        left_label.pack(anchor=tk.W, pady=5)
        
        left_scroll = tk.Scrollbar(left_frame)
        left_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.left_text = tk.Text(left_frame, bg=self.entry_bg, fg=self.entry_fg,
                                wrap=tk.WORD, yscrollcommand=left_scroll.set,
                                font=("Consolas", 10))
        self.left_text.pack(fill=tk.BOTH, expand=True)
        left_scroll.config(command=self.left_text.yview)
        
        # Right text area
        right_frame = tk.Frame(compare_frame, bg=self.bg_color)
        right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=5)
        
        right_label = tk.Label(right_frame, text="Modified Text (Right)", bg=self.bg_color,
                              fg=self.fg_color, font=("Arial", 12, "bold"))
        right_label.pack(anchor=tk.W, pady=5)
        
        right_scroll = tk.Scrollbar(right_frame)
        right_scroll.pack(side=tk.RIGHT, fill=tk.Y)
        
        self.right_text = tk.Text(right_frame, bg=self.entry_bg, fg=self.entry_fg,
                                 wrap=tk.WORD, yscrollcommand=right_scroll.set,
                                 font=("Consolas", 10))
        self.right_text.pack(fill=tk.BOTH, expand=True)
        right_scroll.config(command=self.right_text.yview)
        
        # Configure tags for highlighting
        self.left_text.tag_configure('removed', background='#ff6b6b', foreground='white')
        self.right_text.tag_configure('added', background='#51cf66', foreground='white')
    
    def compare_texts(self):
        """Compare the two texts and highlight differences"""
        left_content = self.left_text.get('1.0', tk.END)
        right_content = self.right_text.get('1.0', tk.END)
        
        # Clear previous highlights
        self.left_text.tag_remove('removed', '1.0', tk.END)
        self.right_text.tag_remove('added', '1.0', tk.END)
        
        # Use difflib to find differences
        left_lines = left_content.splitlines()
        right_lines = right_content.splitlines()
        
        diff = difflib.ndiff(left_lines, right_lines)
        
        removals = 0
        additions = 0
        
        # Process diff results
        left_idx = 0
        right_idx = 0
        
        for line in diff:
            if line.startswith('- '):
                # Removed from left
                removals += 1
                start = f"{left_idx + 1}.0"
                end = f"{left_idx + 1}.end"
                self.left_text.tag_add('removed', start, end)
                left_idx += 1
            elif line.startswith('+ '):
                # Added to right
                additions += 1
                start = f"{right_idx + 1}.0"
                end = f"{right_idx + 1}.end"
                self.right_text.tag_add('added', start, end)
                right_idx += 1
            elif line.startswith('  '):
                # No change
                left_idx += 1
                right_idx += 1
        
        # Calculate similarity
        similarity = difflib.SequenceMatcher(None, left_content, right_content).ratio()
        similarity_percent = round(similarity * 100, 2)
        
        # Update statistics
        stats_text = f"Removals: {removals} | Additions: {additions} | Similarity: {similarity_percent}%"
        self.stats_label.config(text=stats_text)
    
    def clear_text_areas(self):
        """Clear both text areas"""
        self.left_text.delete('1.0', tk.END)
        self.right_text.delete('1.0', tk.END)
        self.stats_label.config(text="")
    
    def create_title_bar(self, title, show_back=False):
        """Create custom title bar"""
        self.title_bar = tk.Frame(self.root, bg=self.header_bg, height=30)
        self.title_bar.pack(fill=tk.X)
        
        # Back button
        if show_back:
            back_btn = tk.Button(self.title_bar, text="< Back", bg=self.header_bg,
                                fg=self.fg_color, relief=tk.FLAT, padx=10,
                                command=self.go_back)
            back_btn.pack(side=tk.LEFT, padx=5)
        
        # Title
        title_label = tk.Label(self.title_bar, text=title, bg=self.header_bg,
                              fg=self.fg_color, font=("Arial", 10, "bold"))
        title_label.pack(side=tk.LEFT, padx=10)
        
        # Window controls
        close_btn = tk.Button(self.title_bar, text="X", bg=self.header_bg, fg=self.fg_color,
                            relief=tk.FLAT, width=3, command=self.root.quit)
        close_btn.pack(side=tk.RIGHT)
        
        maximize_btn = tk.Button(self.title_bar, text="[ ]", bg=self.header_bg, fg=self.fg_color,
                                relief=tk.FLAT, width=3, command=self.toggle_fullscreen)
        maximize_btn.pack(side=tk.RIGHT)
        
        minimize_btn = tk.Button(self.title_bar, text="_", bg=self.header_bg, fg=self.fg_color,
                                relief=tk.FLAT, width=3, command=lambda: self.root.iconify())
        minimize_btn.pack(side=tk.RIGHT)
    
    def toggle_fullscreen(self):
        """Toggle fullscreen mode"""
        if self.root.state() == 'zoomed':
            self.root.state('normal')
        else:
            self.root.state('zoomed')
    
    def go_back(self):
        """Go back to startup screen"""
        self.df = None
        self.current_file = None
        self.root.state('normal')
        self.root.geometry("500x300")
        self.center_window(500, 300)
        self.show_startup_screen()
    
    def clear_window(self):
        """Clear all widgets from window"""
        for widget in self.root.winfo_children():
            widget.destroy()

root = TkinterDnD.Tk() if DRAG_DROP_AVAILABLE else tk.Tk()
app = FileViewerApp(root)
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/root/anaconda3/envs/ml/lib/python3.12/tkinter/__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_73434/2470994336.py", line 591, in show_text_compare_screen
    self.root.state('zoomed')  # Fullscreen
    ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/anaconda3/envs/ml/lib/python3.12/tkinter/__init__.py", line 2296, in wm_state
    return self.tk.call('wm', 'state', self._w, newstate)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
_tkinter.TclError: bad argument "zoomed": must be normal, iconic, or withdrawn
